In [ ]:
!pip install -U langchain-community
!pip install chromadb

In [ ]:
#Load the Data

from langchain.document_loaders import WebBaseLoader

#add website data
URL = ["https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/",
       "https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/"]

#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

In [ ]:
#Text Split - Chunking

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)


In [ ]:
#Embeddings

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = "access token"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking,embeddings)


In [ ]:
#Step-1 Retrieval
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "what is recurrent neural network?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)


[Document(metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks'}, page_content='Recurrent Neural Networks (RNNs) are a type of neural network that is able to process sequential data, such as time series, text, or audio. This makes them well-suited for tasks such as language translation, speech recognition, and time series prediction.'), Document(metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US',

In [ ]:
#Step-2 Augment

prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [ ]:
#Step-3 Generation

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.5,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Recurrent Neural Networks (RNNs) are a type of neural network that is able to process sequential data, such as time series, text, or audio. This makes them well-suited for tasks such as language translation, speech recognition, and time series prediction.

Comment
    More info
Advertise with us



Next Article




Types of Recurrent Neural Networks (RNN) in Tensorflow














 

jaintarun 





 Follow 















Improve






Article Tags : 


AI-ML-DS


Deep Learning

Recurrent Neural Networks are the hardest to train. Since this Neural Network is pruned to Vanishing Gradients. To overcome this issue, many researchers worked to develop a better version of RNN. Some solution provided to Vanishing gradients is to include

Question: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Ple